# Clean Dataset 3 'AGE' using KNN

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## EXPLORE THE DATASET

In [4]:
# Load the dataset
df = pd.read_csv('dataset3.csv')
df.head()

,ID,KIDSDRIV,BIRTH,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,...,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CLM_AMT,CAR_AGE,CLAIM_FLAG,URBANICITY
0,63581743,0,16MAR39,60.0,0,11.0,"$67,349",No,$0,z_No,...,Minivan,yes,"$4,461",2,No,3,$0,18.0,0,Highly Urban/ Urban
1,132761049,0,21JAN56,43.0,0,11.0,"$91,449",No,"$257,252",z_No,...,Minivan,yes,$0,0,No,0,$0,1.0,0,Highly Urban/ Urban
2,921317019,0,18NOV51,48.0,0,11.0,"$52,881",No,$0,z_No,...,Van,yes,$0,0,No,2,$0,10.0,0,Highly Urban/ Urban
3,727598473,0,05MAR64,35.0,1,10.0,"$16,039",No,"$124,191",Yes,...,z_SUV,no,"$38,690",2,No,3,$0,10.0,0,Highly Urban/ Urban
4,450221861,0,05JUN48,51.0,0,14.0,NaN,No,"$306,251",Yes,...,Minivan,yes,$0,0,No,0,$0,6.0,0,Highly Urban/ Urban


In [7]:
# Check for missing values in the 'AGE' column
missing_values = df['AGE'].isna().sum()
print(f"Missing values in 'AGE': {missing_values}")

Missing values in 'CAR_AGE': 7


## CLEAN THE DATASET

In [9]:
# Function to clean and convert columns to numeric
def clean_numeric_column(df, column):
    df[column] = df[column].replace(r'[\$,]', '', regex=True).astype(float)

# Columns to convert
columns_to_convert = ['INCOME', 'HOME_VAL', 'BLUEBOOK', 'OLDCLAIM', 'CLM_AMT']

for column in columns_to_convert:
    clean_numeric_column(df, column)

# After cleaning, confirm that data types are correct
print(df.dtypes)

ID              int64
KIDSDRIV        int64
BIRTH          object
AGE           float64
HOMEKIDS        int64
YOJ           float64
INCOME        float64
PARENT1        object
HOME_VAL      float64
MSTATUS        object
GENDER         object
EDUCATION      object
OCCUPATION     object
TRAVTIME        int64
CAR_USE        object
BLUEBOOK      float64
TIF             int64
CAR_TYPE       object
RED_CAR        object
OLDCLAIM      float64
CLM_FREQ        int64
REVOKED        object
MVR_PTS         int64
CLM_AMT       float64
CAR_AGE       float64
CLAIM_FLAG      int64
URBANICITY     object
dtype: object


In [11]:
# Split into numerical and categorical columns
numerical_columns = ['KIDSDRIV', 'AGE', 'HOMEKIDS', 'YOJ', 'TRAVTIME', 'TIF', 'MVR_PTS', 'INCOME', 'HOME_VAL', 'BLUEBOOK', 'CLM_AMT', 'CAR_AGE']
categorical_columns = ['PARENT1', 'MSTATUS', 'GENDER', 'EDUCATION', 'CAR_USE', 'CAR_TYPE', 'RED_CAR', 'REVOKED', 'URBANICITY', 'OCCUPATION']

# Ensure the lists are correctly defined
print("Numerical columns:", numerical_columns)
print("Categorical columns:", categorical_columns)

Numerical columns: ['KIDSDRIV', 'AGE', 'HOMEKIDS', 'YOJ', 'TRAVTIME', 'TIF', 'MVR_PTS', 'INCOME', 'HOME_VAL', 'BLUEBOOK', 'CLM_AMT', 'CAR_AGE']
Categorical columns: ['PARENT1', 'MSTATUS', 'GENDER', 'EDUCATION', 'CAR_USE', 'CAR_TYPE', 'RED_CAR', 'REVOKED', 'URBANICITY', 'OCCUPATION']


## CHECK FOR CORRELATIONS IN THE DATASET

In [13]:
correlation_matrix = df[numerical_columns].corr()

# Print correlation values for AGE
print(correlation_matrix['AGE'].sort_values(ascending=False))

AGE         1.000000
HOME_VAL    0.209600
INCOME      0.186202
CAR_AGE     0.178592
BLUEBOOK    0.166290
YOJ         0.140878
TRAVTIME   -0.001569
TIF        -0.001890
CLM_AMT    -0.049034
MVR_PTS    -0.075732
KIDSDRIV   -0.078427
HOMEKIDS   -0.447787
Name: AGE, dtype: float64


In [15]:
# Group by categorical columns and print mean AGE
for col in categorical_columns:
    print(df.groupby(col)['AGE'].mean())

PARENT1
No     45.873381
Yes    37.908140
Name: AGE, dtype: float64
MSTATUS
Yes     45.447462
z_No    43.918958
Name: AGE, dtype: float64
GENDER
M      45.483701
z_F    44.282671
Name: AGE, dtype: float64
EDUCATION
<High School     42.223545
Bachelors        44.683091
Masters          47.358999
PhD              49.159529
z_High School    43.180000
Name: AGE, dtype: float64
CAR_USE
Commercial    44.530095
Private       45.016290
Name: AGE, dtype: float64
CAR_TYPE
Minivan        44.906424
Panel Truck    46.492380
Pickup         44.220090
Sports Car     45.397109
Van            45.613043
z_SUV          44.186394
Name: AGE, dtype: float64
RED_CAR
no     44.695605
yes    45.187268
Name: AGE, dtype: float64
REVOKED
No     44.982180
Yes    43.799206
Name: AGE, dtype: float64
URBANICITY
Highly Urban/ Urban      45.020793
z_Highly Rural/ Rural    44.109126
Name: AGE, dtype: float64
OCCUPATION
Clerical         41.769521
Doctor           50.638629
Home Maker       44.728894
Lawyer           48.34